# Image Segmentation on Aerial Images

I present here my best model for the task. In a nutshell it is a U-Net architecture with pre-trained Encoder and a fairly standard Decoder stage and some custom losses.

First of all I install from a github repository the model used for the encoding part of the network, an EfficientNet B7.

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

After that there's the standard input pipeline consisting in: 
* setting the seeds
* checking for the GPU
* import of the data with a training/validation set splitting and creation of the dataset. 

Since now I'm using a pretrained network I give as input an additional parameter `preprocess_input` which is the correct function to use for image pre-processing for the network we're going to use in the training stage.

In [ ]:
from efficientnet.tfkeras import preprocess_input
import tensorflow as tf 
import data_utils as utils

# HYPERPARAMETERS
SEED = 42
utils.set_seed(SEED)
    
utils.check_gpu()

BS = 8
val_split = 0.15
apply_data_augmentation = False
preprocess_input = preprocess_input 
train_dataset, train_img_gen, valid_dataset, valid_img_gen = utils.train_val_dataset(val_split, apply_data_augmentation,BS,SEED,preprocess_input)

# Model Creation 

Now let's analyze the creation of the model. As explained earlier, I used EfficientNet B7 as a backbone. Due to the rather small number of parameters I've tried to re-train it completely rather than freeze the weights in order to achieve better performances. 

So I set the whole model to be `trainable` and then I extracted: 
* the output of the last layer in order to attach the decoder
* the skip connection points from the network 

This network is very complicated but I report here the points I used to do skip connections between encoder and decoder

```python
std_skip_connections = [
    'block2a_expand_activation',
    'block3a_expand_activation',
    'block4a_expand_activation',
    'block6a_expand_activation',
]
```


For the decoder I opted for a 5-layers-deep network based on the following building block:
* **UpSampling**
* **Concatenation** (skip connection)
* **Conv2D layer**
* **Batch Normalization**
* **Activation Layer** with **ReLU** activation function
* **Conv2D layer**
* **Batch Normalization**
* **Activation Layer** with **ReLU** activation function

The number of filters of the two `Conv2D` in the convolution block of the decode from 256 (at the very bottom) to 16 as powers of 2. 

The last layer is standard, a single `1x1` filters with a `sigmoid` activation function. 

---

Notice that I also insert some `Dropout` layers in the Decoder to avoid overfitting. In particular I choose the maxmun amount of dropout (`0.5`) in the deepest (with more filters) layers and as we go shallower the dropout factor reduce. 

In [ ]:
import tensorflow as tf
from efficientnet.tfkeras import EfficientNetB7

DIM = 256
batchnorm = True

# ENCODER
base = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(DIM, DIM, 3),classes=1)
base.trainable = True
base_out = base.output

std_skip_connections = [
    'block2a_expand_activation',
    'block3a_expand_activation',
    'block4a_expand_activation',
    'block6a_expand_activation',
]

def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True, seed=42):
    
    ker_init = tf.keras.initializers.he_normal(seed=seed)
    
        # LAYER 1
    x = tf.keras.layers.Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer=ker_init, padding="same")(input_tensor)
    if batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
        
        # LAYER 2
    x = tf.keras.layers.Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer=ker_init, padding="same")(x)
    if batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    return x

u6 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear') (base_out)
u6 = tf.keras.layers.concatenate([u6, base.get_layer(std_skip_connections[-1]).output], axis=-1)
u6 = tf.keras.layers.Dropout(0.5)(u6)
c6 = conv2d_block(u6, n_filters=256, kernel_size=3, batchnorm=batchnorm, seed=SEED)

u7 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear') (c6)
u7 = tf.keras.layers.concatenate([u7, base.get_layer(std_skip_connections[2]).output], axis=-1)
u7 = tf.keras.layers.Dropout(0.5)(u7)
c7 = conv2d_block(u7, n_filters=128, kernel_size=3, batchnorm=batchnorm, seed=SEED)

u8 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear') (c7)
u8 = tf.keras.layers.concatenate([u8, base.get_layer(std_skip_connections[1]).output], axis=-1)
u8 = tf.keras.layers.Dropout(0.5)(u8)
c8 = conv2d_block(u8, n_filters=64, kernel_size=3, batchnorm=batchnorm, seed=SEED)

u9 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear') (c8)
u9 = tf.keras.layers.concatenate([u9, base.get_layer(std_skip_connections[0]).output], axis=-1)
u9 = tf.keras.layers.Dropout(0.25)(u9)
c9 = conv2d_block(u9, n_filters=32, kernel_size=3, batchnorm=batchnorm, seed=SEED)

u10 = tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear') (c9)
u10 = tf.keras.layers.Dropout(0.25)(u10)
c10 = conv2d_block(u10, n_filters=16, kernel_size=3, batchnorm=batchnorm, seed=SEED)

output_ = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid') (c10)

model = tf.keras.models.Model(inputs=[base.input], outputs=[output_])
                                  
# model.summary()

## Loss Function Definition 

For the loss function I opted for using different losses summed together. The underlying idea is that every one of them will help the model to learn more by focusing in different aspects:

* **Dice Loss**: measure the similaity between the prediction and the ground thruth (`gt`)
* **Focal Loss**: alternative to `binary_crossentropy` that strongly penalizes misclassification since it increases the weights for badly-classified pixels and decreases the weights for well-classified pixels. 
* **Boundary Loss**: Loss function that weights the border of the buildings forcing the network to be very accurate in the edge detection. 

---

The boundary detection loss has been implemented inspiring to this [post](https://www.groundai.com/project/boundary-loss-for-remote-sensing-imagery-semantic-segmentation/1#S4.F1). It uses a pixel-wise pooling with a sliding window of size (3,3) on the inverted mask to gather the boundaries and with another pooling operation it weights each pixel with regards to the presence of multiple boundaries in a (5,5) neighborhood.  


In [ ]:
import tensorflow.keras.backend as backend
from tensorflow_core.keras.backend import pool2d as pool

SMOOTH = 1e-5

def get_reduce_axes(per_image=None):
    axes = [1, 2] if backend.image_data_format() == 'channels_last' else [2, 3]
    if not per_image:
        axes.insert(0, 0)
    return axes

def average(x, per_image=False):
    if per_image:
        x = backend.mean(x, axis=0)
    return backend.mean(x)

def f_score(gt, pr, beta=1, smooth=SMOOTH):

    axes = get_reduce_axes()

    tp = backend.sum(gt * pr, axis=axes)
    fp = backend.sum(pr, axis=axes) - tp
    fn = backend.sum(gt, axis=axes) - tp

    score = ((1 + beta ** 2) * tp + smooth) / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth)
    score = average(score)
    return score

def binary_focal_loss(gt, pr, gamma=2.0, alpha=0.25):
    # clip to prevent NaNs
    pr = backend.clip(pr, backend.epsilon(), 1.0 - backend.epsilon())

    loss_1 = - gt * (alpha * backend.pow((1 - pr), gamma) * backend.log(pr))
    loss_0 = - (1 - gt) * ((1 - alpha) * backend.pow((pr), gamma) * backend.log(1 - pr))
    loss = backend.mean(loss_0 + loss_1)
    return loss

def bf1_score(gt, pr):
    gt = backend.cast(gt, 'float32')
    pr = backend.cast(pr, 'float32')

    b_true = pool(1-gt, pool_size=(3,3), strides=(1,1), padding='same') - (1-gt)
    b_pred = pool(1-pr, pool_size=(3,3), strides=(1,1), padding='same') - (1-pr)

    map_ext_true = pool(b_true, pool_size=(5,5), strides=(1,1), padding='same', pool_mode='avg')
    map_ext_pred = pool(b_pred, pool_size=(5,5), strides=(1,1), padding='same', pool_mode='avg')

    P = tf.reduce_sum(b_pred*map_ext_true)/tf.reduce_sum(b_pred)
    R = tf.reduce_sum(b_true*map_ext_pred)/tf.reduce_sum(b_true)
    return (2 * P * R)/(P + R)
    
def total_loss(y_true, y_pred):
    bf1_loss = 1 - bf1_score(y_true, y_pred)
    foc_loss = binary_focal_loss(y_true, y_pred)
    dic_loss = 1 - f_score(y_true, y_pred)
    return dic_loss + foc_loss + bf1_lossbf1_loss

## Model Training 

The training procedure is standard. I defined the `IoU` metrics and compiled the model with the previously defined loss function and the `Adam` optimizer with an initial learning rate of `5e-5`. 

For the fitting procedure I used three different callbacks:
* **`ReduceLROnPlateau`**: that reduce the learning rate of `factor` after `patience` epochs of non-descending `val_loss`. 
* **`EarlyStopping`**: in order to stop the fit if the `val_loss` does not improve after `patience` epochs. 
* **`ModelCheckpoint`**: to save the best model. 



In [ ]:
import tensorflow.keras.backend as backend
from tensorflow_core.keras.backend import pool2d as pool

def IoU(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32) 
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    return intersection / union

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=5e-5),
    loss=total_loss,
    metrics=[IoU])

callback = []
callback.append(tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=3))
callback.append(tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10,
    restore_best_weights=True))
callback.append(tf.keras.callbacks.ModelCheckpoint(
    './best_model.h5', 
    save_weights_only=True, 
    save_best_only=True, 
    mode='min'))

model.fit(x=train_dataset,
        epochs=150,
        steps_per_epoch=len(train_img_gen),
        validation_data=valid_dataset,
        validation_steps=len(valid_img_gen),
        callbacks=callback)

## Optimizations and Creation of the CSV file 

After the training procedure I performed a cross-validation on the validation set for the threshold (for discernign 0s and 1s) in order to maximize the prediction accuracy. 
I did scan all the range between `0.05` and `0.95` and re-defined the metrics to take an additional parameter (the new threshold) as input. After that, in a `for` loop, I used the `model.evaluate()` method on the validation set and saved all the metrics performaces in a vector whose is then used for selecting the proper thershold. 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline

n_cv = 10
thresholds = np.linspace(0.25,0.75,n_cv)
m = np.zeros((n_cv,1))

def IoUscore(th):
    def IoU(y_true, y_pred):
        y_pred = tf.cast(y_pred > th, tf.float32) 
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

        return intersection / union
    return IoU

for i in range(n_cv):
    metric = IoUscore(th=thresholds[i])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-5),loss=total_loss,metrics=[metric])
    m[i] = model.evaluate(valid_dataset, steps=len(valid_img_gen))[1]

best_tm = thresholds[np.argmax(m)]
print('OPT THRESHOLD: ',best_tm)
print('BEST VAL: ', np.max(m))
plt.figure()
plt.plot(m)

At this point all that's left to do is the creation the output CSV file.
During the various try of training I did notice that the masks' edges were rough and very noisy with some spots of misclassified pixels in the center of some roofs. 
In order to fix these problems I used some morphological transformation on the predicted (and thresholded) masks: 
* **closing**: is a dilation followed by erosion operation that helps closing small holes inside the foreground objects, or small black points on the object. 
* **opening**: is a erosion followed by dilation. It is useful in removing noise in the image. 

This procedure helpd a lot in improving the model accuracy.

The `sub_csv_postprcess` function is very similar to the the `sub_csv` function that I used for making predictions and generating the CSV file in the other models, the only difference is that it emplyes the morphological transformation that I explained before. 

In [ ]:
utils.sub_csv_postprocess(model,preprocess_input,best_tm)

## Comments

The model is very long to train (on Kaggle platform, using GPU) taking the whole time window for GPU usage (9h). 

The model can be further improved using: 
* Data augmentation
* Ensembles method (which I was not able to test because of the long training time and the limit amount of GPU in Kaggle)
* Additional post-processing (e.g. polygonizing the predicted masks)